# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
base_url = 'https://sandbox.iexapis.com/stable'
api_url = f"{base_url}/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
print(data['symbol'])
price = data['latestPrice']
print(price)

market_cap = data['marketCap']
print(market_cap / 1000000000000)

AAPL
148.88
2.57180455923


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = [ 'Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [7]:
final_dataframe.append(
    pd.Series(
        [symbol, price, market_cap, 'N/A'],
        index = my_columns
    ),
    ignore_index = True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,148.88,2571804559230,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
def download_data():
    dataframe = pd.DataFrame(columns = my_columns)
    for symbol in stocks['Ticker']:
        api_url = f"{base_url}/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
        data = requests.get(api_url).json()
        price = data['latestPrice']
        market_cap = data['marketCap']
        buy_count = 'N/A'
        return dataframe.append(
            pd.Series(
                [symbol, price, market_cap, buy_count],
                index = my_columns
            ),
            ignore_index = True
        )

In [9]:
final_dataframe = download_data()
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,175.22,52336741782,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [10]:
def chunks(iterable, n = 1):
    """Yield successive n-sized chunks from iterable"""
    length = len(iterable)
    for ndx in range(0, length, n):
        yield iterable[ndx: ndx + n] #min(ndx + n, length)]

In [28]:
def download_data_batched():
    symbol_groups = list(chunks(stocks['Ticker'], 100))
    symbol_strings = []

    for i in range(0, len(symbol_groups)):
        symbol_strings.append(",".join(symbol_groups[i]))

    dataframe = pd.DataFrame(columns = my_columns)
        
    for symbols in symbol_strings:
        api_url = f"{base_url}/stock/market/batch?symbols={symbols}&types=quote&token={IEX_CLOUD_API_TOKEN}"

        data = requests.get(api_url).json()

        for symbol in symbols.split(','):
            price = data[symbol]['quote']['latestPrice']
            market_cap = data[symbol]['quote']['marketCap']
            buy_count = 'N/A'

            dataframe = dataframe.append(
                pd.Series(
                    [symbol, price, market_cap, buy_count],
                    index = my_columns
                ),
                ignore_index = True
            )

    return dataframe

final_dataframe = download_data_batched()
print(final_dataframe)

    Ticker  Stock Price Market Capitalization Number of Shares to Buy
0        A       169.90           52949096789                     N/A
1      AAL        19.30           12507827789                     N/A
2      AAP       212.71           14066161989                     N/A
3     AAPL       148.72         2503607137692                     N/A
4     ABBV       120.87          218487546079                     N/A
..     ...          ...                   ...                     ...
500    YUM       140.97           40697625392                     N/A
501    ZBH       149.95           30997962814                     N/A
502   ZBRA       587.76           31173383262                     N/A
503   ZION        54.15            9033493962                     N/A
504    ZTS       212.81           98540757316                     N/A

[505 rows x 4 columns]


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [42]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
except ValueError:
    print('Portfolio size should be an integer.')
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

In [53]:
portfolio_size = val / len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    stock_price = final_dataframe.loc[i, 'Stock Price']
    buy_count = math.floor(portfolio_size / stock_price)
    final_dataframe.loc[i, 'Number of Shares to Buy'] = buy_count

# num_shares = math.floor(portfolio_size / 500)
# print(final_dataframe)

    Ticker  Stock Price Market Capitalization Number of Shares to Buy
0        A       169.90           52949096789                      11
1      AAL        19.30           12507827789                     102
2      AAP       212.71           14066161989                       9
3     AAPL       148.72         2503607137692                      13
4     ABBV       120.87          218487546079                      16
..     ...          ...                   ...                     ...
500    YUM       140.97           40697625392                      14
501    ZBH       149.95           30997962814                      13
502   ZBRA       587.76           31173383262                       3
503   ZION        54.15            9033493962                      36
504    ZTS       212.81           98540757316                       9

[505 rows x 4 columns]


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: